<a href="https://colab.research.google.com/github/ghr8635/E2E-DriveAI-ROS2-based-Modular-Framework-for-Autonomous-Vehicle-Control/blob/main/self_built_point_pillar_architecture_(simple_data).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install open3d

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.7/399.7 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 116.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 83.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 72.1 MB/s eta 0:00:00
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.6.10
    Uninstalling widgetsnbextension-3.6.10:
      Successfully uninstalled widgetsnbextension-3.6.10
  Attempting uninstall: werkzeug
    Found existing installation: Werkzeug 3.1.2
    Uninstalling Werkzeug-3.1.2:
      Successfully uninstalled Werkzeug-3.1.2
  Attempting uninstall: ipywidgets
    Found existing installation: ipywidgets 7.7.1
    Uninstalling ipywidgets-7.7.1:
      Successfully uninstalled ipywidgets-7

In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from torch.utils.data import Dataset, DataLoader

**Define the PointPillars Feature Extraction Model**

In [10]:
class PillarFeatureEncoder(nn.Module):
    def __init__(self, in_channels, out_channels, num_points_per_pillar, grid_x, grid_y):
        super(PillarFeatureEncoder, self).__init__()
        self.num_points_per_pillar = num_points_per_pillar
        self.grid_x, self.grid_y = grid_x, grid_y
        # Simple linear layer to encode pillar features
        self.fc = nn.Linear(in_channels, out_channels)

    def forward(self, pillars):
        # Flatten and encode features
        pillars = self.fc(pillars)  # Shape: (batch, grid_x, grid_y, num_points, out_channels)
        return pillars.mean(dim=2)  # Reduce across point dimension to get (batch, grid_x, grid_y, out_channels)

class BackboneNetwork(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(BackboneNetwork, self).__init__()
        # Simple 2D CNN layers for feature extraction
        self.conv1 = nn.Conv2d(in_channels, 64, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1)
        self.conv3 = nn.Conv2d(128, out_channels, kernel_size=3, stride=2, padding=1)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = self.conv3(x)  # Shape: (batch, out_channels, grid_x//4, grid_y//4)
        return x

class PointPillarsFeatureExtractor(nn.Module):
    def __init__(self, in_channels=4, out_channels=256, num_points_per_pillar=32, grid_x=100, grid_y=100):
        super(PointPillarsFeatureExtractor, self).__init__()
        self.pfe = PillarFeatureEncoder(in_channels, out_channels, num_points_per_pillar, grid_x, grid_y)
        self.backbone = BackboneNetwork(out_channels, out_channels)

    def forward(self, x):
        pillar_features = self.pfe(x)  # Shape: (batch, grid_x, grid_y, out_channels)
        pillar_features = pillar_features.permute(0, 3, 1, 2)  # Shape: (batch, out_channels, grid_x, grid_y)
        feature_map = self.backbone(pillar_features)
        return feature_map

**Prepare a Simple Dataset for Training which mimics pcd data**

In [11]:
class SyntheticPointCloudDataset(Dataset):
    def __init__(self, num_samples, grid_x=100, grid_y=100, num_points_per_pillar=32, in_channels=4):
        self.num_samples = num_samples
        self.grid_x, self.grid_y = grid_x, grid_y
        self.num_points_per_pillar = num_points_per_pillar
        self.in_channels = in_channels

    def __len__(self):
        return self.num_samples

    def __getitem__(self, idx):
        # Generate random pillars with shape: (grid_x, grid_y, num_points, in_channels)
        pillars = np.random.rand(self.grid_x, self.grid_y, self.num_points_per_pillar, self.in_channels).astype(np.float32)
        return torch.tensor(pillars)

# Parameters
num_samples = 100
dataset = SyntheticPointCloudDataset(num_samples)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)


**Training Loop (Optional, to learn feature extraction patterns)**

In [ ]:
# Instantiate model, optimizer, and criterion
model = PointPillarsFeatureExtractor()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.MSELoss()

# Simple Training Loop
epochs = 5
for epoch in range(epochs):
    for batch_idx, pillars in enumerate(dataloader):
        optimizer.zero_grad()

        # Forward pass
        features = model(pillars)  # Extracted features, shape: (batch, out_channels, grid_x//4, grid_y//4)

        # Dummy target: here we're using the output itself as target for demonstration
        target = features.clone().detach()  # This is just for illustrative purposes
        loss = criterion(features, target)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        print(f"Epoch [{epoch+1}/{epochs}], Batch [{batch_idx+1}/{len(dataloader)}], Loss: {loss.item():.4f}")


Epoch [1/5], Batch [1/25], Loss: 0.0000
Epoch [1/5], Batch [2/25], Loss: 0.0000
Epoch [1/5], Batch [3/25], Loss: 0.0000
Epoch [1/5], Batch [4/25], Loss: 0.0000
Epoch [1/5], Batch [5/25], Loss: 0.0000
Epoch [1/5], Batch [6/25], Loss: 0.0000
Epoch [1/5], Batch [7/25], Loss: 0.0000
Epoch [1/5], Batch [8/25], Loss: 0.0000
Epoch [1/5], Batch [9/25], Loss: 0.0000
Epoch [1/5], Batch [10/25], Loss: 0.0000
Epoch [1/5], Batch [11/25], Loss: 0.0000
Epoch [1/5], Batch [12/25], Loss: 0.0000
Epoch [1/5], Batch [13/25], Loss: 0.0000
Epoch [1/5], Batch [14/25], Loss: 0.0000
Epoch [1/5], Batch [15/25], Loss: 0.0000
Epoch [1/5], Batch [16/25], Loss: 0.0000
Epoch [1/5], Batch [17/25], Loss: 0.0000
Epoch [1/5], Batch [18/25], Loss: 0.0000
Epoch [1/5], Batch [19/25], Loss: 0.0000
Epoch [1/5], Batch [20/25], Loss: 0.0000
Epoch [1/5], Batch [21/25], Loss: 0.0000
Epoch [1/5], Batch [22/25], Loss: 0.0000
Epoch [1/5], Batch [23/25], Loss: 0.0000
Epoch [1/5], Batch [24/25], Loss: 0.0000
Epoch [1/5], Batch [25/25

**Saving Model**

In [ ]:
# Define the file path where you want to save the model
model_save_path = "/content/drive/MyDrive/ROS2-Modular-Framework-for-End-to-End-Autonomous-Vehicle-Control-from-Raw-Sensor-Data/self_built_point_pillar.pth"

# After training is complete, save the model
torch.save(model.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")


Model saved to /content/drive/MyDrive/ROS2-Modular-Framework-for-End-to-End-Autonomous-Vehicle-Control-from-Raw-Sensor-Data/self_built_point_pillar.pth


**Note: Upto this point the point pillar based feature extracting model is trained with simply generated dataset and saved. The next part is for inference where a pcd file is processed and provided to the model for inference.**

In [4]:
import open3d as o3d
import numpy as np

# Load the PCD file
pcd = o3d.io.read_point_cloud('/content/drive/MyDrive/ROS2-Modular-Framework-for-End-to-End-Autonomous-Vehicle-Control-from-Raw-Sensor-Data/lidar_01__2023-06-02-21-28-09-321.pcd')

# Convert the point cloud into a NumPy array (N, 3), where N is the number of points
points = np.asarray(pcd.points)

print(f"Loaded point cloud with {points.shape[0]} points")


Loaded point cloud with 65536 points


In [7]:
def preprocess_point_cloud(points, grid_x, grid_y, voxel_size=0.1):
    """
    Convert 3D point cloud to pillar representation.

    Args:
    - points: (N, 3) numpy array where N is the number of points.
    - grid_x: Number of pillars along the X axis.
    - grid_y: Number of pillars along the Y axis.
    - voxel_size: Size of each voxel (grid cell).

    Returns:
    - pillars: The preprocessed point cloud as pillar features.
    """
    x_min, y_min = np.min(points[:, 0]), np.min(points[:, 1])
    x_max, y_max = np.max(points[:, 0]), np.max(points[:, 1])

    # Create grid for the 2D space
    grid_x_range = np.linspace(x_min, x_max, grid_x)
    grid_y_range = np.linspace(y_min, y_max, grid_y)

    # Initialize pillar features
    pillars = np.zeros((grid_x, grid_y, 32, 4))  # (grid_x, grid_y, max_points_per_pillar, feature_dim)

    for point in points:
        # Calculate the 2D grid coordinates for the point
        x_idx = int((point[0] - x_min) / voxel_size)
        y_idx = int((point[1] - y_min) / voxel_size)

        # Ensure indices are within the grid bounds
        x_idx = min(x_idx, grid_x - 1)
        y_idx = min(y_idx, grid_y - 1)

        # Add the point features (x, y, z, intensity) to the pillar
        pillar = pillars[x_idx, y_idx]

        # If there is an empty space in the pillar (0 entries), insert the point
        empty_slot_idx = np.where(pillar[:, 0] == 0)[0]

        if len(empty_slot_idx) > 0:
            # Fill the first empty slot (use the first one)
            pillar_idx = empty_slot_idx[0]
            pillars[x_idx, y_idx, pillar_idx] = np.append(point, 1)  # Add intensity as 1 for now
        else:
            # If no empty slots, replace the point in the pillar (you could add a logic to choose the best point)
            pillar_idx = np.argmin(np.linalg.norm(pillar[:, :3], axis=1))  # Choose the point closest to the origin of the pillar
            pillars[x_idx, y_idx, pillar_idx] = np.append(point, 1)  # Replace with the new point's features

    return pillars

# Set grid resolution and voxel size
grid_x, grid_y = 100, 100  # Resolution of your grid
preprocessed_pillars = preprocess_point_cloud(points, grid_x, grid_y)


**Loading the Model for Inference**

In [12]:
import torch

# Initialize the model (assuming you already have the model architecture and weights loaded)
model = PointPillarsFeatureExtractor()

# Load the saved weights
model_save_path = '/content/drive/MyDrive/ROS2-Modular-Framework-for-End-to-End-Autonomous-Vehicle-Control-from-Raw-Sensor-Data/self_built_point_pillar.pth'
model.load_state_dict(torch.load(model_save_path))

# Set the model to evaluation mode
model.eval()

# Convert the preprocessed point cloud into a torch tensor
input_tensor = torch.tensor(preprocessed_pillars, dtype=torch.float32).unsqueeze(0)  # Add batch dimension

# Perform inference (no gradient calculation since we're not training)
with torch.no_grad():
    feature_map = model(input_tensor)

# Process the feature_map for your task (if needed)
print("Feature map extracted:")
print(feature_map.shape)  # Check the output shape


<ipython-input-12-c32fd5471caa>:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_save_path))


Feature map extracted:
torch.Size([1, 256, 25, 8])
